In [ ]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime, timedelta
import re


In [ ]:
report_by_years = dict()
input_by_years = dict()
reports = dict()
report_dates = list()
dates_to_check = list()

pattern = re.compile(r'^\s*(\d{2}\s*.\s*\d{2}\s*.\s*\d{4})\s*$')
new_entry=True

years = ["06", "07", "08", "09", "10", "11", "12"]

path = '/usr/local/bin/chromedriver'
browser = webdriver.Chrome()


In [ ]:
def remove_duplicate_dates(data):
    new_data = ""
    lines = data.split('\n')
    dates_to_check.clear()
    prev_date = datetime.now().strftime("%d.%m.%Y")
    for line in lines:
        if line == "":
            continue
        if re.search(pattern, line):
            line = line.replace(" ", "")
            if prev_date == line:
                data.replace(line, " ")
            else:
                new_data = new_data + line + '\n'
                date_time_obj = datetime.strptime(line, '%d.%m.%Y')
                if date_time_obj.weekday() == 4:
                    date_time_obj += timedelta(days=2)
                date_time_obj += timedelta(days=1)
                dates_to_check.append(date_time_obj.strftime("%d.%m.%Y"))
            prev_date = line
        else:
            new_data += line + "\n"
    print(new_data)
    return new_data


In [ ]:
def get_data(link, year):
    # access the link
    browser.get(link)

    data = browser.find_element_by_xpath('//*[@id="mainBody"]/section/div/div[2]/div/div[3]/article').text
    report_file = open("reports_"+year+".txt", "w+")
    data = remove_duplicate_dates(data)
    dates_to_check_file = open("dates_to_check_"+year+".txt", "w+")
    #print(data)
    #print(dates_to_check)
    report_file.write(data)
    dates_to_check_file.write("\n".join(dates_to_check))
    dates_to_check_file.close()
    report_file.close()


In [ ]:
for year in years:
    year = "20"+year
    link = "http://investor.turkishairlines.com/en/announcements/disclosures/detail/"+year+"-public-disclosures"
    get_data(link, year)
